In [27]:
import tensorflow
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.layers import Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences

import re
import pandas as pd
import numpy as np

import simplemma
from stop_words import get_stop_words
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

In [28]:
stop_words = get_stop_words('uk')

In [29]:
#попередня обробка тексту
def ClearText(text):
    #переведення до нижнього регістру всіх слів
    cleartext = text.lower()
    #print(cleartext)
    #прибирання пустих рядків та розрив рядків
    cleartext = re.sub('\-\s\r\n\s{1,}|\-\s\r\n|\r\n', '', cleartext) 
    #залишаємо лише слова, прибираємо пунктуацію та числа
    cleartext = re.sub('[.,:;_%©?*,!@#$%^&()\d]|[+=]|[[]|[]]|[/]|"|\s{2,}|-', ' ', cleartext) #deleting symbols  
    #cleartext = cleartext.translate(remove_digits)
    cleartext = cleartext.replace("\\", "")
    cleartext = cleartext.rstrip()
    #прибираємо зайві пробіи
    cleartext = re.sub(" +", " ", cleartext)
    #ділимо речення на список слів, розбиваємо по пробілам
    cleartext = re.split(" ", cleartext)
    #прибираємо стопслова
    cleartext = [word for word in cleartext if word not in stop_words]
    #прибираємо слова, довжина який менше 3 букв
    cleartext = [word for word in cleartext if len(word) > 3]
    #лематизація слів
    cleartext = [simplemma.lemmatize(word, lang='uk') for word in cleartext]
    return ' '.join(cleartext)

In [40]:
df = pd.read_excel('/home/irina/Study/NLP/NLP/NLP_all.xlsx')
df = df.sample(frac=1) 

MAX_NB_WORDS = 50000
MAX_SEQUENCE_LENGTH = 100
EMBEDDING_DIM = 50

df['ClearText'] = df.apply(lambda x: ClearText(x['Comment']), axis=1)

tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
X = tokenizer.texts_to_sequences(df['ClearText'].values)
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)

Y = pd.get_dummies(df['Category']).values

In [41]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.10)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(1046, 100) (1046, 2)
(117, 100) (117, 2)


In [42]:

# !pip install keras
# !pip install tensorflow


model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(10,dropout=0.4, recurrent_dropout=0.4))
model.add(Dense(2, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())



Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 100, 50)           2500000   
                                                                 
 spatial_dropout1d_3 (Spatia  (None, 100, 50)          0         
 lDropout1D)                                                     
                                                                 
 lstm_3 (LSTM)               (None, 10)                2440      
                                                                 
 dense_3 (Dense)             (None, 2)                 22        
                                                                 
Total params: 2,502,462
Trainable params: 2,502,462
Non-trainable params: 0
_________________________________________________________________
None


In [43]:
epochs = 15
batch_size = 16

history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1,callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])


Epoch 1/15
59/59 [==============================] - 10s 101ms/step - loss: 0.6633 - accuracy: 0.6387 - val_loss: 0.6547 - val_accuracy: 0.6381
Epoch 2/15
59/59 [==============================] - 6s 108ms/step - loss: 0.6466 - accuracy: 0.6546 - val_loss: 0.6567 - val_accuracy: 0.6381
Epoch 3/15
59/59 [==============================] - 9s 151ms/step - loss: 0.6475 - accuracy: 0.6546 - val_loss: 0.6554 - val_accuracy: 0.6381
Epoch 4/15
59/59 [==============================] - 7s 123ms/step - loss: 0.6475 - accuracy: 0.6546 - val_loss: 0.6557 - val_accuracy: 0.6381


In [44]:
accr = model.evaluate(X_test,Y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

4/4 [==============================] - 0s 12ms/step - loss: 0.6366 - accuracy: 0.6667
Test set
  Loss: 0.637
  Accuracy: 0.667
